In [1]:
# July 2023
# Implementing VoxNet:
# you can see the VoxNet's architecture at https://github.com/AutoDeep/VoxNet

# 1) Voxeling Modelnet10

In [2]:
dataset_addr = '/home/sbn/Downloads/datasets/data_/ModelNet10/'

In [3]:
import torch
from torch.utils.data import DataLoader
import numpy as np




# import sys
# sys.path.insert(0, dataset_addr)
from modelnet10 import ModelNet10

CLASSES = {
    0: 'bathtub',
    1: 'chair',
    2: 'dresser',
    3: 'night_stand',
    4: 'sofa',
    5: 'toilet',
    6: 'bed',
    7: 'desk',
    8: 'monitor',
    9: 'table'
}
N_CLASSES = len(CLASSES)



train_dataset = ModelNet10(data_root= dataset_addr, 
                           n_classes=N_CLASSES, 
                           idx2cls=CLASSES, 
                           split='train')

train_dataloader = DataLoader(train_dataset, 
                              batch_size=1, 
                              shuffle=True)




{'bathtub': 0, 'chair': 1, 'dresser': 2, 'night_stand': 3, 'sofa': 4, 'toilet': 5, 'bed': 6, 'desk': 7, 'monitor': 8, 'table': 9}


# Testing voxel

In [4]:
for sample in train_dataloader:
    voxel, cls_idx = sample['voxel'], sample['cls_idx']
    print(voxel.shape)
    break

torch.Size([1, 1, 32, 32, 32])


# VoxNet

![image](images/VoxNet.png)

In [5]:
import torch
from torch import nn

In [6]:
VoxNet = nn.Sequential(
                nn.Conv3d(in_channels=1, out_channels=32, kernel_size=(5,5,5), stride=2),
                nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), stride=1),
                nn.AvgPool3d(kernel_size=(2,2,2), stride=2),
                nn.Flatten(start_dim=1),
                nn.Linear(6912, 128),
                nn.Linear(128, N_CLASSES), 
                nn.ReLU()
)

In [7]:
# voxel

In [8]:
VoxNet(voxel.float()).shape

torch.Size([1, 10])

In [14]:
optimizer = torch.optim.SGD(VoxNet.parameters(), lr=0.003)

In [15]:
loss = nn.CrossEntropyLoss()

In [18]:
print("Start training ...")
VoxNet.train()

for epoch in range(30):
    overall_loss = 0
    for batch_idx, sample in enumerate(train_dataloader):
        x,y =  sample['voxel'], sample['cls_idx']
        
        optimizer.zero_grad()
        y_hat = VoxNet(x.float()).float()
        ll = loss(y_hat, y.long())
        overall_loss += ll.item()
        ll.backward()
        optimizer.step()
        
#     print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*batch_size))
    print(overall_loss)
    
print("Finish!!")

Start training ...


KeyboardInterrupt: 

In [35]:
VoxNet.eval()
for sample in train_dataloader:
    sample_x, sample_y = sample['voxel'], sample['cls_idx']
    sample_y_hat = torch.argmax(VoxNet(sample_x.float()))
    print(sample_y)
    print(sample_y_hat)
#     print(voxel.shape)
    break

tensor([9])
tensor(9)
